In [85]:
from dataclasses import dataclass
import casadi as ca
import numpy as np
import time
import math
from typing import Dict, Optional
import logging
import datetime
import timeit
from tqdm import tqdm

from ocean_navigation_simulator.data_sources import OceanCurrentSource
from ocean_navigation_simulator.data_sources.SolarIrradiance.SolarIrradianceSource import SolarIrradianceSource
from ocean_navigation_simulator.data_sources.SeaweedGrowth.SeaweedGrowthSource import SeaweedGrowthSource
from ocean_navigation_simulator.utils import units
from ocean_navigation_simulator.environment.PlatformState import PlatformState
from ocean_navigation_simulator.environment.PlatformState import SpatialPoint
from ocean_navigation_simulator.environment.ArenaFactory import ArenaFactory
from ocean_navigation_simulator.environment.NavigationProblem import NavigationProblem
from ocean_navigation_simulator.controllers.hj_planners.HJReach2DPlanner import HJReach2DPlanner
from ocean_navigation_simulator.utils import units
import matplotlib.pyplot as plt
import os
os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl')
print(os.getcwd())

/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl


+ Create 4 Platforms as usual using the `PlatformState` class 

In [86]:
x_0_1 = PlatformState(lon=units.Distance(deg=-82.5), lat=units.Distance(deg=23.7),
                    date_time=datetime.datetime(2021, 11, 24, 12, 0, tzinfo=datetime.timezone.utc), id=int(0))
x_0_2 = PlatformState(lon=units.Distance(deg=-82.6), lat=units.Distance(deg=23.8),
                    date_time=datetime.datetime(2021, 11, 24, 12, 0, tzinfo=datetime.timezone.utc), id=int(1))
x_0_3 = PlatformState(lon=units.Distance(deg=-82.4), lat=units.Distance(deg=23.6),
                    date_time=datetime.datetime(2021, 11, 24, 12, 0, tzinfo=datetime.timezone.utc), id=int(2))
x_0_4 = PlatformState(lon=units.Distance(deg=-82.4), lat=units.Distance(deg=23.7),
                    date_time=datetime.datetime(2021, 11, 24, 12, 0, tzinfo=datetime.timezone.utc), id=int(2))

Create a first implementation version of PlatformStateSet

In [87]:
from typing import List
import dataclasses
from dataclasses import astuple
@dataclasses.dataclass
class PlatformStateSet_v1:
    states:List[PlatformState]

    def __array__(self):
        states_list = [np.array(state) for state in self.states]
        return np.array(states_list).T #columns: number of platforms

    def __len__(self):
        return len(self.states)

    def __getitem__(self, platform_id):
        return np.array(self.states[platform_id])

Second version of platform state set, states directly stored as numpy array (->useful for Casadi) and implement methods to convert to list (for example if we would run HJ)

In [88]:
@dataclasses.dataclass
class PlatformStateSet_v2:
    states: np.array

    def get_list_platform_state(self):
        return [PlatformState.from_numpy(self.states[:,id]) for id in range(self.states.shape[1])] 

    def get_platform_state_from_id(self, id):
        return PlatformState.from_numpy(self.states[:,id])

Time it to get a numpy array (row = states, col = platform id)

In [89]:
x_set_v1 = PlatformStateSet_v1(states=[x_0_1, x_0_2])
%timeit np.array(x_set_v1)
x_set_v2 = PlatformStateSet_v2(states = np.array(x_set_v1))
%timeit x_set_v2.get_list_platform_state()

18.8 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
21.6 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [90]:
x_set_v1 = PlatformStateSet_v1(states=[x_0_1, x_0_2,x_0_3, x_0_4])
%timeit np.array(x_set_v1)
x_set_v2 = PlatformStateSet_v2(states = np.array(x_set_v1))
%timeit x_set_v2.get_list_platform_state()

34 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
41.8 µs ± 3.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Scale this up in terms of platforms:

Define a function to generate (randomly) platform samples

In [91]:
def rand_platforms(nb_platforms):
    lon = np.random.uniform(low=-83.5, high=-83, size=(nb_platforms,))
    lat = np.random.uniform(low=23, high=23.5, size=(nb_platforms,))
    id = np.arange(start=1, stop=nb_platforms+1, step=1)
    t = np.repeat(datetime.datetime(2021, 11, 24, 12, 0, tzinfo=datetime.timezone.utc), nb_platforms)
    return lon,lat,id,t


In [92]:
nb_platforms = 50
lon,lat,id,t = rand_platforms(nb_platforms)
platforms_list = [PlatformState(lon=units.Distance(deg=lon[k]), lat=units.Distance(deg=lat[k]), date_time=t[k], id=int(id[k])) \
                 for k in range(nb_platforms)]

In [93]:
x_set_v1 = PlatformStateSet_v1(states=platforms_list)
%timeit np.array(x_set_v1)
x_set_v2 = PlatformStateSet_v2(states = np.array(x_set_v1))
%timeit x_set_v2.get_list_platform_state()

362 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
492 µs ± 55.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Compare to the time if we only extract on platform state

In [94]:
plat_id= np.random.randint(low=1,high=nb_platforms)
%timeit x_set_v2.get_platform_state_from_id(plat_id)

10 µs ± 488 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Now compare speed of casadi vectorization compared to single platform: 

First start to initialize the scenario and to set up the ocean source environment

In [95]:
from ocean_navigation_simulator.data_sources.OceanCurrentField import OceanCurrentField
from ocean_navigation_simulator.data_sources.SeaweedGrowthField import SeaweedGrowthField
from ocean_navigation_simulator.data_sources.SolarIrradianceField import SolarIrradianceField
from typing import Dict, Optional, Union, Tuple, List, AnyStr, Literal, Callable
import yaml

scenario_name = 'gulf_of_mexico_HYCOM_hindcast_local'
with open(f'config/arena/{scenario_name}.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
casadi_cache_dict=config['casadi_cache_dict']
platform_dict=config['platform_dict']
ocean_dict= config['ocean_dict']
use_geographic_coordinate_system=config['use_geographic_coordinate_system']
spatial_boundary=config['spatial_boundary']

ocean_field = OceanCurrentField(
    casadi_cache_dict=casadi_cache_dict,
    hindcast_source_dict=ocean_dict['hindcast'],
    forecast_source_dict=ocean_dict['forecast'],
    use_geographic_coordinate_system=use_geographic_coordinate_system)

ocean_source = ocean_field.hindcast_data_source

INFO:arena.ocean_field:DataField: Create Hindcast Source (7.8s)
INFO:arena.ocean_field:DataField: Forecast is the same as Hindcast for OceanCurrents.


Define a PlatformActionSet class and define a function to generate random actions

In [96]:
from ocean_navigation_simulator.environment.Platform import Platform, PlatformAction
@dataclasses.dataclass
class PlatformActionSet:
    action_set: List[PlatformAction]

    def __array__(self):
        action_list = [np.array(action) for action in self.action_set]
        return np.array(action_list).T #columns: number of platforms

def gen_rand_action(nb_platforms):
    mag = np.random.uniform(low=0, high=99, size=(nb_platforms,))
    dir = np.random.uniform(low=0, high=2*np.pi, size=(nb_platforms,))
    return mag, dir

In [97]:
nb_platforms = 10
lon,lat,id,t = rand_platforms(nb_platforms)
platforms_list = [PlatformState(lon=units.Distance(deg=lon[k]), lat=units.Distance(deg=lat[k]), date_time=t[k], id=int(id[k])) \
                 for k in range(nb_platforms)]
mag, dir = gen_rand_action(nb_platforms)
actions_list = [PlatformAction(magnitude=mag[k], direction=dir[k]) for k in range(nb_platforms)]
platforms_set = PlatformStateSet_v1(states=platforms_list)
action_set = PlatformActionSet(action_set =actions_list)

Initialize the interpolation around a single platform for now (we will redefine the interpolation region also with multiple platforms) but for this need to change `update_casadi_dynamics` and `convert_to_x_y_time_bounds` in `DataSource.py`

In [98]:
import casadi as ca
from ocean_navigation_simulator.utils import units
ocean_source.update_casadi_dynamics(platforms_list[0])
u_max = units.Velocity(mps=platform_dict['u_max_in_mps']) # normally defined as attribute of the class Platform
dt_in_s = platform_dict['dt_in_s']

Implement the simplified CasADi dynamics function

In [99]:
def get_casadi_dynamics(nb_platforms, ocean_source, u_max):
    state_row_dict = {"lon":0, "lat":1, "time":2, "battery":3, "seaweed":4 }
    action_row_dict = {"thrust": 0, "angle":1}
    sym_dt              = ca.MX.sym('dt')           # in s
    sym_inputs_states   = ca.MX.sym('inputs_states', 5, nb_platforms)
    sym_inputs_ctrl     = ca.MX.sym('inputs_ctrl', 2, nb_platforms)

    sym_lon_degree = sym_inputs_states[state_row_dict["lon"],:]
    sym_lat_degree = sym_inputs_states[state_row_dict["lat"],:]
    sym_time = sym_inputs_states[state_row_dict["time"],:]
    sym_battery = sym_inputs_states[state_row_dict["battery"],:]
    sym_seaweed_mass = sym_inputs_states[state_row_dict["seaweed"],:]
    sym_u_thrust = sym_inputs_ctrl[action_row_dict["thrust"],:]
    sym_u_angle = sym_inputs_ctrl[action_row_dict["angle"],:]

    # Get currents
    u_curr = ocean_source.u_curr_func(ca.vertcat(sym_time,sym_lat_degree, sym_lon_degree))
    v_curr = ocean_source.v_curr_func(ca.vertcat(sym_time, sym_lat_degree, sym_lon_degree))
    sym_lon_delta_meters_per_s = ca.cos(sym_u_angle)*sym_u_thrust*u_max.mps + u_curr
    sym_lat_delta_meters_per_s = ca.sin(sym_u_angle)*sym_u_thrust*u_max.mps + v_curr
    sym_lon_delta_deg_per_s = 180 * sym_lon_delta_meters_per_s / math.pi / 6371000 / ca.cos(math.pi * sym_lat_degree / 180)
    sym_lat_delta_deg_per_s = 180 * sym_lat_delta_meters_per_s / math.pi / 6371000

     # Equations for next states using the intermediate variables from above
    sym_lon_next = sym_lon_degree + sym_dt * sym_lon_delta_deg_per_s
    sym_lat_next = sym_lat_degree + sym_dt * sym_lat_delta_deg_per_s
    sym_time_next = sym_time + sym_dt
    # F_next = ca.Function('F_x_next', [ca.vertcat(sym_lon_degree, sym_lat_degree, sym_time, sym_battery, sym_seaweed_mass), ca.vertcat(sym_u_thrust, sym_u_angle), sym_dt],
    #                     [ca.vertcat(sym_lon_next, sym_lat_next, sym_time_next)])

    F_next = ca.Function('F_x_next', [sym_inputs_states, sym_inputs_ctrl, sym_dt ], [ca.vertcat(sym_lon_next, sym_lat_next, sym_time_next)] )
    return F_next

Single platform time to run a casadi step (e.g. for the first platform in the list)

In [100]:
F_x_next = get_casadi_dynamics(1, ocean_source, u_max)
%timeit F_x_next(np.array(platforms_set.states[0]), np.array(action_set.action_set[0]), dt_in_s)

69.9 µs ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Multiple platform vectorized casadi dynamics:

In [101]:
F_x_next = get_casadi_dynamics(nb_platforms, ocean_source, u_max)
%timeit F_x_next(np.array(platforms_set), np.array(action_set), dt_in_s)

219 µs ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Increase the number of platforms

In [102]:
nb_platforms = 100
lon,lat,id,t = rand_platforms(nb_platforms)
platforms_list = [PlatformState(lon=units.Distance(deg=lon[k]), lat=units.Distance(deg=lat[k]), date_time=t[k], id=int(id[k])) \
                 for k in range(nb_platforms)]
mag, dir = gen_rand_action(nb_platforms)
actions_list = [PlatformAction(magnitude=mag[k], direction=dir[k]) for k in range(nb_platforms)]
platforms_set = PlatformStateSet_v1(states=platforms_list)
action_set = PlatformActionSet(action_set =actions_list)

In [103]:
F_x_next = get_casadi_dynamics(nb_platforms, ocean_source, u_max)
%timeit F_x_next(np.array(platforms_set), np.array(action_set), dt_in_s)

1.4 ms ± 76.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
